In [1]:
import pencilbox as pb
import pandas as pd
import numpy as np
import time
from datetime import date, datetime, timedelta
from statsmodels.tsa.seasonal import seasonal_decompose
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import register_matplotlib_converters

CON_REDSHIFT = pb.get_connection("[Warehouse] Redshift")

In [2]:
register_matplotlib_converters()
sns.set_style("darkgrid")

plt.rc("figure", figsize=(16, 12))
plt.rc("font", size=12)

In [3]:
def read_sql_query(sql, con):
    start = time.time()
    df = pd.read_sql_query(sql, con)
    end = time.time()
    if (end - start) > 60:
        print("Time: ", (end - start) / 60, "min")
    else:
        print("Time: ", end - start, "s")
    return df

In [ ]:
order_query = f"""
select 
    
    cart_checkout_ts_ist::date as Date, 
    
    case when extract(day from cart_checkout_ts_ist::date) <= 7 
    then 1
    when extract(day from cart_checkout_ts_ist::date) <= 14 
    then 2
    when extract(day from cart_checkout_ts_ist::date) <= 21 
    then 3
    when extract(day from cart_checkout_ts_ist::date) <= 28 
    then 4
    else 5 
    end as wom,
    
    extract(day from cart_checkout_ts_ist::date) as dom,
    
    date_part(dow,cart_checkout_ts_ist::date) as dow,
    
    p.l2_category, 
    
    sum(procured_quantity) as qty
    
from dwh.fact_sales_order_item_details fsoid 
join dwh.dim_product p on fsoid.dim_product_key = p.product_key 

where order_current_status = 'DELIVERED'
and cart_checkout_ts_ist between date('2023-03-01')-180 and date('2023-03-01')
and is_internal_order = false 
and l2_category != 'Freebie'
--and city_name = 'HR-NCR'
--and l2_category = 'Atta'

group by 1,2,3,4,5"""

order_data = read_sql_query(order_query, CON_REDSHIFT)

In [ ]:
order_query = f"""
with
sub_order_details as
    (select 
        date(install_ts + interval '5.5 Hours') as order_date,
        id as sub_order_id,
        order_id
        
            from lake_oms_bifrost.oms_suborder
            
                where (install_ts between (current_date - 30 || ' 00:00:00')::timestamp - interval '5.5 Hours'
                    and (current_date || ' 23:59:59')::timestamp - interval '5.5 Hours')
    ),

invoice_details as
    (select 
        order_date,
        pi.id as invoice_id,
        grofers_order_id as sub_order_id,
        cart_id,
        order_id,
        outlet_id,
        external_id,
        invoice_type_id
        
            from lake_pos.pos_invoice pi
            
                join
                    sub_order_details od on od.sub_order_id = pi.grofers_order_id
                
                join
                    lake_oms_bifrost.oms_order oo on oo.id = od.order_id and oo.current_status = 'DELIVERED'
                    and oo.install_ts >= (current_date - 30) and oo.type not ilike '%%internal%%'
                    
                    where pi.created_at >= (current_date - 30)
                        and invoice_type_id in (1,7,2,8)
    ),
    
invoice_item_details as
    (select 
        order_date,
        invoice_type_id,
        cart_id,
        order_id,
        id.outlet_id,
        rpc.brand_id,
        item_id,
        rpc.name,
        rpc.variant_id,
        upc_id,
        quantity,
        selling_price,
        variant_mrp
        
            from lake_pos.pos_invoice_product_details pd
            
                join
                    invoice_details id on id.invoice_id = pd.invoice_id
                
                join
                    lake_rpc.product_product rpc on rpc.variant_id = pd.variant_id
                    
                    where pd.pos_timestamp >= (current_date - 30)
    ),

sales as
    (select 
        a.order_date as date_,
        a.order_id,
        a.outlet_id,
        a.item_id,
        coalesce(return_quantity,0.0) as return_quantity,
        coalesce(return_selling_price,0.0) as return_selling_price,
        coalesce(return_variant_mrp,0.0) as return_variant_mrp,
        
        sum(quantity) as sales_quantity,
        sum(quantity * selling_price) as sales_selling_price,
        sum(quantity * variant_mrp) as sales_variant_mrp
        
            from invoice_item_details a
            
                left join
                    (select order_id, item_id,
                        sum(quantity) as return_quantity,
                        sum(quantity * selling_price) as return_selling_price,
                        sum(quantity * variant_mrp) as return_variant_mrp
                            from invoice_item_details
                                where invoice_type_id in (2,8)
                                    group by 1,2
                    ) b on b.order_id = a.order_id and b.item_id = a.item_id
                    
                    where invoice_type_id in (1,7)
                    
                        group by 1,2,3,4,5,6,7
    ),

final_sales as
    (select 
        date_,
        cl.name as city_name,
        s.outlet_id,
        s.item_id, 
        cd.name as item_name,
        l2 as category,
        sum(sales_quantity - return_quantity) as sales_quantity,
        sum(sales_selling_price - return_selling_price) as sales_value,
        sum(sales_variant_mrp - return_variant_mrp) as sales_mrp
        
            from sales s
            
                join
                    lake_rpc.item_category_details cd on cd.item_id = s.item_id
                
                join
                    lake_retail.console_outlet rco on rco.id = s.outlet_id and rco.business_type_id in (7)
                join
                    lake_retail.console_location cl on cl.id = rco.tax_location_id
                    
                    group by 1,2,3,4,5,6
    )
    
        select * from final_sales
"""

order_data = read_sql_query(order_query, CON_REDSHIFT)

In [ ]:
# order_data.to_pickle('seasonality.pkl')

In [ ]:
# order_data = pd.read_pickle('seasonality.pkl')

In [ ]:
order_data.head(10)

In [ ]:
l2_list = order_data['l2_category'].unique()

In [ ]:
len(l2_list)

In [ ]:
weekly_seasonality_df = pd.DataFrame()
monthly_seasonality_df = pd.DataFrame()

for i in tqdm(l2_list): 
    data_orig = order_data.loc[order_data['l2_category']==i]
    data_orig = data_orig.sort_values(by=['date'])
    data_orig = data_orig.reset_index()
    
    if len(data_orig)>=60:

        weekly_components = seasonal_decompose(data_orig['qty'], period=7, model='additive')
        monthly_components = seasonal_decompose(data_orig['qty'], period=30, model='additive')

        # adjusted = (data_orig['qty'].values/weekly_components.seasonal)
        # /monthly_components.seasonal
        data_orig['weekly_seasonal']=weekly_components.seasonal
        data_orig['monthly_seasonal']=monthly_components.seasonal

        # data_orig['seasonal']=monthly_components.seasonal
        monthly_seasonality_df = monthly_seasonality_df.append(data_orig.groupby(['l2_category', 'dom'])['monthly_seasonal'].mean().reset_index())
        weekly_seasonality_df = weekly_seasonality_df.append(data_orig.groupby(['l2_category', 'dow'])['weekly_seasonal'].mean().reset_index())

        
monthly_seasonality_df = monthly_seasonality_df[['l2_category','dom','monthly_seasonal']].reset_index()
weekly_seasonality_df = weekly_seasonality_df[['l2_category','dow','weekly_seasonal']].reset_index()

In [ ]:
monthly_seasonality_df

In [ ]:
seasonal_decomp = seasonal_decompose(order_data['qty'], period=7, model='additive')
seasonal_decomp.plot()

In [ ]:
product_query = f"""
select 
    
    cart_checkout_ts_ist::date as Date, 
    
    case when extract(day from cart_checkout_ts_ist::date) <= 7 
    then 1
    when extract(day from cart_checkout_ts_ist::date) <= 14 
    then 2
    when extract(day from cart_checkout_ts_ist::date) <= 21 
    then 3
    when extract(day from cart_checkout_ts_ist::date) <= 28 
    then 4
    else 5 
    end as wom,
    
    extract(day from cart_checkout_ts_ist::date) as dom,
    
    date_part(dow,cart_checkout_ts_ist::date) as dow,
    
    p.l2_category, 
    
    p.product_id,
    
    sum(procured_quantity) as qty
    
from dwh.fact_sales_order_item_details fsoid 
join dwh.dim_product p on fsoid.dim_product_key = p.product_key 

where order_current_status = 'DELIVERED'
and cart_checkout_ts_ist between date('2023-03-01')-180 and date('2023-03-01')
and is_internal_order = false 
and l2_category != 'Freebie'
--and city_name = 'HR-NCR'
--and l2_category = 'Atta'

group by 1,2,3,4,5,6"""

product_data = read_sql_query(product_query, CON_REDSHIFT)

In [ ]:
!pip install duckdb
import duckdb


In [ ]:
main_query = """

select a.date, a.wom, a.dom, a.dow, a.l2_category, a.product_id, a.qty, a.qty*1.00/(coalesce(b.monthly_seasonal,1)*coalesce(c.weekly_seasonal,1)) as adjusted

from product_data a 

left join monthly_seasonality_df b on a.dom = b.dom and a.l2_category = b.l2_category

left join weekly_seasonality_df c on a.dow = c.dow and a.l2_category = c.l2_category

"""


In [ ]:
final_data = duckdb.query(main_query).to_df()

In [ ]:
final_data

In [ ]:
product_list = final_data['product_id'].unique()

In [ ]:
final_df = pd.DataFrame()

for i in tqdm(product_list): 
    
    data_orig = final_data[final_data['product_id']==i]

    data_orig['rolling_median']=data_orig['adjusted'].shift(1).rolling(21).median()
    data_orig['rolling_75']=data_orig['adjusted'].shift(1).rolling(21).quantile(0.75)
    data_orig['rolling_25']=data_orig['adjusted'].shift(1).rolling(21).quantile(0.25)
    data_orig['IQR'] = data_orig['rolling_75'] - data_orig['rolling_25']

    factor = 1.5

    # data_orig['zscore'] = (data_orig['adjusted']-data_orig['rolling_median'])/(data_orig['rolling_75']-data_orig['rolling_25'])

    # data_orig['spike'] = np.where(data_orig['zscore']>=3, 1,0)


    data_orig['spike'] = np.where((data_orig['adjusted']>(factor*data_orig['IQR'] + data_orig['rolling_75'])) | (data_orig['adjusted']<(data_orig['rolling_25']-factor*data_orig['IQR'])),1,0)

    # data_orig['zscore'] = (data_orig['adjusted']/data_orig['rolling_mean'])
    # data_orig['spike'] = np.where(abs(data_orig['zscore']) >= 0.85, np.where(abs(data_orig['zscore']) <= 1.15, 0, 1),1) 

    final_df = final_df.append(data_orig)

In [ ]:
# x = seasonality_df['dom']
# y = seasonality_df['seasonal']

# fig, ax = plt.subplots()
# ax.plot_date(x, y, linestyle='--')

In [ ]:
# monthly_components.plot()

In [ ]:
# seasonality_df[seasonality_df['l2_category'].str.contains('Oil')]['l2_category'].unique()

In [ ]:
# seasonality_df[seasonality_df['l2_category']=='Sunflower Oil']

In [ ]:
# seasonality_df[seasonality_df['l2_category']=='Oil & Ghee']

In [ ]:
# seasonality_df[seasonality_df['l2_category']=='Oil & Ghee']

In [ ]:
# !pip install duckdb
# import duckdb

# main_query = """select a.date, 
#                         a.wom, 
#                         a.dow	l2_category	qty from df"""
# final_data = duckdb.query(main_query).to_df()

In [ ]:
# data_orig['adjusted'] = data_orig['qty']/data_orig['seasonal']

In [ ]:
# data_orig['rolling_mean']=data_orig['adjusted'].shift(1).rolling(21).ewm(alpha=0.1).median()
# data_orig['rolling_std']=data_orig['adjusted'].shift(1).rolling(21).quantile(0.75)-data_orig['adjusted'].shift(1).rolling(21).quantile(0.25)

# data_orig['zscore'] = (data_orig['adjusted']-data_orig['rolling_mean'])/data_orig['rolling_std']
# data_orig['spike'] = np.where(abs(data_orig['zscore']) >= 1.5, 1, 0) 

# spike_df = data_orig.loc[data_orig['spike']==1]

# spike_df

In [ ]:
data_orig['rolling_median']=data_orig['adjusted'].shift(1).rolling(21).median()
data_orig['rolling_75']=data_orig['adjusted'].shift(1).rolling(21).quantile(0.75)
data_orig['rolling_25']=data_orig['adjusted'].shift(1).rolling(21).quantile(0.25)
data_orig['IQR'] = data_orig['rolling_75'] - data_orig['rolling_25']

factor = 1.5

# data_orig['zscore'] = (data_orig['adjusted']-data_orig['rolling_median'])/(data_orig['rolling_75']-data_orig['rolling_25'])

# data_orig['spike'] = np.where(data_orig['zscore']>=3, 1,0)


data_orig['spike'] = np.where((data_orig['adjusted']>(factor*data_orig['IQR'] + data_orig['rolling_75'])) | (data_orig['adjusted']<(data_orig['rolling_25']-factor*data_orig['IQR'])),1,0)
                               
# data_orig['zscore'] = (data_orig['adjusted']/data_orig['rolling_mean'])
# data_orig['spike'] = np.where(abs(data_orig['zscore']) >= 0.85, np.where(abs(data_orig['zscore']) <= 1.15, 0, 1),1) 

spike_df = data_orig.loc[data_orig['spike']==1]

spike_df

In [ ]:
# data_orig

In [ ]:
# len(spike_df)*100/len(data_orig)

In [ ]:
# import datetime as dt
# import matplotlib.pyplot as plt
# import matplotlib.dates as mdates

# x = data_orig['date']
# y = data_orig['adjusted']

# fig, ax = plt.subplots()
# ax.plot_date(x, y, linestyle='--')

# for i in range(len(spike_df)):
#     if(spike_df['spike'].iloc[i]>0):
#         ax.annotate('Anomaly', (mdates.date2num(spike_df['date'].iloc[i]), spike_df['adjusted'].iloc[i]), xytext=(15, 15), 
#                 textcoords='offset points', arrowprops=dict(arrowstyle='-|>'), c='r')

# fig.autofmt_xdate()
# plt.show()